In [20]:
import pandas as pd 
import numpy as np 
import torch
from sklearn.model_selection import train_test_split
from utils import sequence_utils
from transformers import BertTokenizer, BertForSequenceClassification


In [25]:
device =  torch.device("mps")
print(device)

mps


In [3]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
MAX_LEN = 160
BATCH_SIZE = 16

In [4]:
PRE_TRAINED_MODEL_NAME = '../ai-models/bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME) 

In [5]:
path = "../datasets/goemotions/data/full_dataset/"
file_1 = "goemotions_1.csv"
file_2 = "goemotions_2.csv"
file_3 = "goemotions_3.csv"
df_1 = pd.read_csv(path+file_1)
df_2 = pd.read_csv(path+file_2)
df_3 = pd.read_csv(path+file_3)
dfs_ = [df_1, df_2, df_3]
_dfs = pd.concat(dfs_)

In [6]:
_dfs.shape 

(211225, 37)

In [7]:
_dfs.head()

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,True,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,False,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1.547965e+09,18,False,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1.546669e+09,2,False,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
dataframe = _dfs.drop(_dfs[_dfs['example_very_unclear'] == True].index)
dataframe.shape

(201202, 37)

In [9]:
dataframe.tail()

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
71219,"Well, I'm glad you're out of all that now. How...",ed89acy,pompompompi,raisedbynarcissists,t3_ac9etw,t1_ed7gt8x,1.546615e+09,2,False,0,...,0,0,0,0,0,0,0,0,0,0
71220,Everyone likes [NAME].,ee6pagw,Senshado,heroesofthestorm,t3_agjf24,t3_agjf24,1.547634e+09,16,False,0,...,1,0,0,0,0,0,0,0,0,0
71221,Well when you’ve imported about a gazillion of...,ef28nod,5inchloser,nottheonion,t3_ak26t3,t3_ak26t3,1.548553e+09,15,False,0,...,0,0,0,0,0,0,0,0,0,0
71222,That looks amazing,ee8hse1,springt1me,shittyfoodporn,t3_agrnqb,t3_agrnqb,1.547684e+09,70,False,1,...,0,0,0,0,0,0,0,0,0,0
71223,The FDA has plenty to criticize. But like here...,edrhoxh,enamedata,medicine,t3_aejqzd,t1_edrgdtx,1.547169e+09,4,False,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
def emotions(row):
    cols_ = row.index[9:]
    tmp = 0
    for col in cols_:
        tmp = tmp + 1
        if row[col] == 1:
            return tmp


In [11]:
dataframe['emotions'] = dataframe.apply(emotions, axis=1)

In [12]:
dataframe

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,emotions
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,...,0,0,0,0,0,0,1,0,0,26
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,False,0,...,0,0,0,0,0,0,0,0,1,28
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1.547965e+09,18,False,0,...,0,0,0,0,0,0,0,0,0,19
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1.546669e+09,2,False,0,...,0,0,0,0,0,0,0,0,1,28
5,Right? Considering it’s such an important docu...,eespn2i,ImperialBoss,TrueReddit,t3_aizyuz,t1_eesoak0,1.548280e+09,61,False,0,...,0,0,0,0,0,0,0,0,0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71219,"Well, I'm glad you're out of all that now. How...",ed89acy,pompompompi,raisedbynarcissists,t3_ac9etw,t1_ed7gt8x,1.546615e+09,2,False,0,...,0,0,0,0,0,0,0,0,0,18
71220,Everyone likes [NAME].,ee6pagw,Senshado,heroesofthestorm,t3_agjf24,t3_agjf24,1.547634e+09,16,False,0,...,0,0,0,0,0,0,0,0,0,19
71221,Well when you’ve imported about a gazillion of...,ef28nod,5inchloser,nottheonion,t3_ak26t3,t3_ak26t3,1.548553e+09,15,False,0,...,0,0,0,0,0,0,0,0,0,6
71222,That looks amazing,ee8hse1,springt1me,shittyfoodporn,t3_agrnqb,t3_agrnqb,1.547684e+09,70,False,1,...,0,0,0,0,0,0,0,0,0,1


In [13]:
df_emotions = dataframe[['text', 'emotions']]
df_emotions 

,text,emotions
0,That game hurt.,26
2,"You do right, if you don't care then fuck 'em!",28
3,Man I love reddit.,19
4,"[NAME] was nowhere near them, he was by the Fa...",28
5,Right? Considering it’s such an important docu...,16
...,...,...
71219,"Well, I'm glad you're out of all that now. How...",18
71220,Everyone likes [NAME].,19
71221,Well when you’ve imported about a gazillion of...,6
71222,That looks amazing,1


In [14]:
df_emotions.isnull().sum()

text        0
emotions    0
dtype: int64

In [15]:
df_emotions.value_counts('emotions')

emotions
28    53528
1     16609
5     15025
4     11525
11     8638
2      8575
16     8154
3      7704
8      7449
10     6567
7      6402
19     5136
6      4969
18     4964
23     4950
21     4838
14     4248
26     3762
27     3354
12     3316
9      2909
15     2428
13     1666
25     1603
20      919
24      790
22      693
17      481
Name: count, dtype: int64

In [16]:
df_train, df_test = train_test_split(df_emotions, test_size=0.2, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)

In [17]:
df_train.shape, df_val.shape, df_test.shape

((160961, 2), (20120, 2), (20121, 2))

In [18]:
train_data_loader = sequence_utils.my_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = sequence_utils.my_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = sequence_utils.my_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [28]:
model = BertForSequenceClassification.from_pretrained(
   PRE_TRAINED_MODEL_NAME, 
    num_labels = 28,                      
    output_attentions = False,
    output_hidden_states = False, 
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../ai-models/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,